In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.callbacks import ModelCheckpoint

/home/alliedtoasters/anaconda3/envs/ann/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Dog Breed Classifier
This project is a response to the <a href = 'https://www.kaggle.com/c/dog-breed-identification'> Kaggle Dog Breed Identification Challenge</a>. The dataset includes photographs of 120 dog breeds and a labeled training set of 10,222 photos.

## Building a Convnet with CPU
Image classification is notoriously computation heavy and it can take a long time to train up a functioning model without the aid of high-performance GPU. I wanted to see what was possible with, and what the limitations are of tackling a challenge like this with a CPU-only machine. These are the steps I took to create a classifier with non-trivial performance:

 - Use a pre-trained convnet: VGG16 trained with ImageNet (includes many dog breeds)
 - Strip "top," flattened layers from model to access raw image features
 - Build new classifier on top of pre-trained VGG16 convolutional layers
 - Final model consists pre-trained VGG16 layers and my custom-trained top layer
 
## Holdout Group and Data Preparation
To prepare the images for processing, I use Scikit-Learn stratified split. To play nice with the keras ImageDataGenerator, I organize the photos (after resizing to 224x224 pixels) into folders by breed class, in one directory for training (80% of photos) and another for validation (20% of photos.) That script can be found <a href='https://github.com/AlliedToasters/dogs/blob/master/make_holdout.ipynb'>here</a>.

## Extracting "Bottleneck" Features
Running the VGG16 convolutional layers on my CPU machine is a very lengthy process (about 4 hours to get through all 10,222 photos.) To avoid running each photo through the convnet more than once, I save the output of each of the images into a numpy array file (.npy). I use the keras ImageDataGenerator to aid in that process by "feeding" the images to the model. This process was taken from <a href='https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html'>this helpful tutorial</a> on the Keras blog. The script I use to extract these features can be found <a href='https://github.com/AlliedToasters/dogs/blob/master/bottleneck_features.ipynb'>here</a>.

## Training Model "Top"
To build a model sensitive to dog breeds, I used the extracted VGG16 features and train up the model. That process is recorded below.<br><br>
The "top" model itself is simply a 256 node-wide layer densely connected to the flattened 

In [2]:
train_features = np.load('features_train.npy')
test_features = np.load('bottleneck_features_validation.npy')

train_labels = np.load('lbl_train.npy')
test_labels = np.load('lbl_test.npy')

In [3]:
inputs = Input(shape=train_features.shape[1:])
x = Flatten()(inputs)
x = Dense(256, activation='relu')(x)
x = Dropout(0.1)(x)
outputs = Dense(120, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [4]:
checkpointer = ModelCheckpoint(filepath='VGG16_topv4.hdf5', verbose=1, save_best_only=True)

In [5]:
model.fit(
    train_features, train_labels,
    epochs=15,
    batch_size=13,
    validation_data=(test_features, test_labels),
    shuffle=True,
    callbacks=[checkpointer],
    verbose=1
)

Train on 8177 samples, validate on 2045 samples
Epoch 1/15
8164/8177 [============================>.] - ETA: 0s - loss: 5.5317 - acc: 0.0102
Epoch 00001: val_loss improved from inf to 4.78289, saving model to VGG16_topv4.hdf5
8177/8177 [==============================] - 61s 7ms/step - loss: 5.5305 - acc: 0.0102 - val_loss: 4.7829 - val_acc: 0.0122
Epoch 2/15
8164/8177 [============================>.] - ETA: 0s - loss: 4.8052 - acc: 0.0115
Epoch 00002: val_loss did not improve
8177/8177 [==============================] - 61s 8ms/step - loss: 4.8050 - acc: 0.0115 - val_loss: 4.8533 - val_acc: 0.0083
Epoch 3/15
8164/8177 [============================>.] - ETA: 0s - loss: 4.7940 - acc: 0.0138
Epoch 00003: val_loss did not improve
8177/8177 [==============================] - 65s 8ms/step - loss: 4.7939 - acc: 0.0138 - val_loss: 4.8070 - val_acc: 0.0103
Epoch 4/15
8164/8177 [============================>.] - ETA: 0s - loss: 4.7882 - acc: 0.0147
Epoch 00004: val_loss improved from 4.78289 to 